In [9]:
# data genrator : generates data for the the ocr
import numpy as np
import tensorflow as tf

import os
import itertools
import codecs
import re
import datetime
import cairocffi as cairo
import editdistance


In [10]:
# character classes and matching regex filter
regex = r'^[a-z ]+$'
alphabet = u'abcdefghijklmnopqrstuvwxyz '

In [11]:
word_file = 'wordlists/wordlist_mono_clean.txt'
file = codecs.open(word_file,mode='r',encoding='utf-8')
        

In [12]:
def get_word():
    line = file.readline()
    word = line.rstrip()
    return word

In [13]:
# Translation of characters to unique integer values
def text_to_labels(text):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


In [14]:
# Reverse translation of numerical classes back to characters
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)

In [15]:
def sparse_tuple_from(sequences, dtype=np.int32):
        """Create a sparse representention of x.

        Args:
            sequences: a list of lists of type dtype where each element is a sequence
        Returns:
            A tuple with (indices, values, shape)
        """
        indices = []
        values = []        
        for n, seq in enumerate(sequences):
            indices.extend(zip([n] * len(seq), range(len(seq))))
            values.extend(seq)
     
        indices = np.asarray(indices, dtype=np.int64)
        values = np.asarray(values, dtype=dtype)
        shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)     
        return indices, values, shape

In [16]:
def is_valid_str(in_str):
    search = re.compile(regex, re.UNICODE).search
    return bool(search(in_str))


In [20]:
# paints the string in a random location the bounding box
# also uses a random font, a slight random rotation,
# and a random amount of speckle noise

def paint_text(text, w, h, rotate=False, ud=False, multi_fonts=False):
    surface = cairo.ImageSurface(cairo.FORMAT_RGB24, w, h)
    with cairo.Context(surface) as context:
        context.set_source_rgb(1, 1, 1)  # White
        context.paint()
        # this font list works in CentOS 7
        if multi_fonts:
            fonts = [
                'Century Schoolbook', 'Courier', 'STIX',
                'URW Chancery L', 'FreeMono']
            context.select_font_face(
                np.random.choice(fonts),
                cairo.FONT_SLANT_NORMAL,
                np.random.choice([cairo.FONT_WEIGHT_BOLD, cairo.FONT_WEIGHT_NORMAL]))
        else:
            context.select_font_face('Courier',
                                     cairo.FONT_SLANT_NORMAL,
                                     cairo.FONT_WEIGHT_BOLD)
        context.set_font_size(15)
        box = context.text_extents(text)
        border_w_h = (4, 4)
        if box[2] > (w - 2 * border_w_h[1]) or box[3] > (h - 2 * border_w_h[0]):
            raise IOError(('Could not fit string into image.'
                           'Max char count is too large for given image width.'))

        # teach the RNN translational invariance by
        # fitting text box randomly on canvas, with some room to rotate
        max_shift_x = w - box[2] - border_w_h[0]
        max_shift_y = h - box[3] - border_w_h[1]
        top_left_x = np.random.randint(0, int(max_shift_x))
        if ud:
            top_left_y = np.random.randint(0, int(max_shift_y))
        else:
            top_left_y = h // 2
        context.move_to(top_left_x - int(box[0]), top_left_y - int(box[1]))
        context.set_source_rgb(0, 0, 0)
        context.show_text(text)

    buf = surface.get_data()
    a = np.frombuffer(buf, np.uint8)
    a.shape = (h, w, 4)
    a = a[:, :, 0]  # grab single channel
    a = a.astype(np.float32) / 255
    #a = np.expand_dims(a, 0)
    if rotate:
        a = image.random_rotation(a, 3 * (w - top_left_x) / w + 1)
    #a = speckle(a)

    return a


In [37]:
img_w = 128
img_h = 64

In [38]:
batch_size = 32

In [39]:
def next_batch():
    
    # image
    X_data = np.ones([batch_size,img_w,img_h,1])
    
    # labels
    labels = []
    
    #input length
    input_length = np.zeros([batch_size, 1])
    
    # label_length
    label_length = np.zeros([batch_size, 1])
    
    for i in range(batch_size):
        
        text = get_word()
        
        X_data[i,0:img_w,:,0] = paint_text(text,w=img_w,h=img_h)[0,:,:].T
        
        label = text_to_labels(text)
        labels.append(label)
        
        
        input_length[i] = img_w//4 -2
        
        label_length[i] = len(label)
        
    labels = sparse_tuple_from(labels)
        
        
    return X_data,labels,input_length
    


In [40]:
imgs,target,lengths = next_batch()

In [41]:
text = get_word()

In [42]:
print(text)

can


In [43]:
imgs.shape

(32, 128, 64, 1)

In [44]:
lengths.shape

(32, 1)

In [45]:
def cnn(inputs):
        """ cnn """
        conv1 = tf.layers.conv2d(inputs=inputs,filters=16,kernel_size=[3,3],padding="same",activation= tf.nn.relu)

        pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)

        conv2 = tf.layers.conv2d(inputs=pool1,filters=16,kernel_size=[3,3],padding="same",activation=tf.nn.relu)

        pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)

        return pool2


In [46]:
def rnn(inputs,seq_len):
    
    
    """ BIdirectinal rnn"""

    #lstm_fw_cell_1 = tf.nn.rnn_cell.BasicLSTMCell(256)

    #lstm_bw_cell_1 = tf.nn.rnn_cell.BasicLSTMCell(256)

    #inter_out,_ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell_1,lstm_bw_cell_1,inputs,seq_len,dtype=tf.float32)

    #inter_out = tf.concat(inter_out,2)
        
    lstm_fw_cell_2 = tf.nn.rnn_cell.BasicLSTMCell(256)

    #lstm_bw_cell_2 = tf.nn.rnn_cell.BasicLSTMCell(256)

    outputs,_ = tf.nn.dynamic_rnn(lstm_fw_cell_2,inputs,seq_len,dtype=tf.float32)

    #outputs = tf.concat(outputs,2)

    return outputs

In [47]:
inputs = tf.placeholder(tf.float32,[batch_size,img_w,img_h,1])
targets = tf.sparse_placeholder(tf.int32,name= "target")
seq_len = tf.placeholder(tf.int32,[1])
lr = tf.placeholder(tf.float32)

In [48]:
n_classes =29

In [49]:
cnn_out = cnn(inputs)
reshaped_cnn_out = tf.reshape(cnn_out,[batch_size,-1,img_h*16])
rnn_in = tf.layers.dense(inputs=reshaped_cnn_out,units=32,activation=tf.nn.relu)
rnn_out =rnn(rnn_in,seq_len)
logits = tf.layers.dense(inputs=rnn_out,units=n_classes,activation=tf.nn.relu)
logits = tf.transpose(logits,(1,0,2))
cost = tf.nn.ctc_loss(targets,logits,seq_len)
loss = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(lr)
train_op = optimizer.minimize(loss)


ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-19-7e0688e810b9>", line 18, in rnn
    outputs,_ = tf.nn.dynamic_rnn(lstm_fw_cell_2,inputs,seq_len,dtype=tf.float32)
  File "<ipython-input-22-e8bc2eeefc96>", line 4, in <module>
    rnn_out =rnn(rnn_in,seq_len)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
init = tf.global_variables_initializer()

In [24]:
learn_rate = 0.001

In [27]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        x_data,labels,in_length = next_batch()
        feed = {inputs:x_data,targets:labels,seq_len:30,lr:learn_rate}
        ls = sess.run(loss,feed_dict=feed)
        
        if i%100 == 0:
            print("loss:"+ls)
        
        

ValueError: Cannot feed value of shape () for Tensor 'Placeholder_1:0', which has shape '(?,)'

In [6]:
text = get_word()

In [7]:
lbl = text_to_labels(text)

In [8]:
lbl

[19, 7, 4]

In [21]:
img = paint_text("chaitu",28*8,28)

In [22]:
img.shape

(28, 224)

In [23]:
img = np.transpose(img)

In [24]:
img.shape

(224, 28)

In [25]:
img = np.expand_dims(img,axis=3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


In [26]:
img.shape

(224, 28, 1)

In [27]:
len(alphabet)

27